In [1]:
import pandas as pd
import numpy as np
import csv
import os

In [5]:
import pandas as pd
import numpy as np
import csv
import os


# read flash.dat to a list of lists
datContent = [i.strip().split() for i in open(r'user_taggedmovies-timestamps.dat').readlines()]
df=pd.DataFrame(datContent[1:],columns=datContent[0])
df.to_csv(r'user_taggedmovies-timestamps.csv')

In [6]:
df=pd.DataFrame(datContent[1:],columns=datContent[0])

In [9]:

def walkFile(file):
    filelist=[]
    for root, dirs, files in os.walk(file):

        for f in files:
            filelist.append(os.path.join(f))

        return filelist
filelist=walkFile(r"hetrec2011-movielens-2k")
filelist

['movies.dat',
 'movie_actors.dat',
 'movie_countries.dat',
 'movie_directors.dat',
 'movie_genres.dat',
 'movie_locations.dat',
 'movie_tags.dat',
 'tags.dat',
 'user_ratedmovies-timestamps.dat',
 'user_ratedmovies.dat',
 'user_taggedmovies-timestamps.dat',
 'user_taggedmovies.dat']

In [ ]:
#convert to csv
datContent = [i.strip().split('	') for i in open(r'movies.dat','r', encoding='mac_roman').readlines()]
df=pd.DataFrame(datContent[1:],columns=datContent[0])
df.to_csv(r'movies.csv')
print(r'movies.csv')
for i in filelist[2:]:
    datContent = [i.strip().split('	') for i in open(r''+i, encoding='mac_roman').readlines()]
    df=pd.DataFrame(datContent[1:],columns=datContent[0])
    i=i.strip('.dat')+'.csv'
    df.to_csv(r''+i)
    print(r''+i)

In [ ]:
movies=pd.read_csv(r'movies.csv')
movies.drop(['Unnamed: 0'],axis=1, inplace=True)
tags=pd.read_csv(r'gs.csv')
tags.drop(['Unnamed: 0'],axis=1, inplace=True)
user_taggedmovies_timestamps=pd.read_csv(r'user_taggedmovies-timestamps.csv')
user_taggedmovies_timestamps.drop(['Unnamed: 0'],axis=1, inplace=True)
user_taggedmovies_timestamps=user_taggedmovies_timestamps.sort_values(by=['timestamp'])
user_taggedmovies=pd.read_csv(r'user_taggedmovies.csv')
user_taggedmovies.drop(['Unnamed: 0'],axis=1, inplace=True)

In [ ]:
#100 most frequent users
frequentUser=np.array(user_taggedmovies.userID.value_counts().index[:5000])
#10 most frequent items
frequentItem=list(user_taggedmovies.movieID.value_counts().index[:10000])
frequentUserData=user_taggedmovies_timestamps[(user_taggedmovies_timestamps['userID'].isin(frequentUser))].sort_values(by=['timestamp'])
freqData=user_taggedmovies_timestamps[(user_taggedmovies_timestamps['userID'].isin(frequentUser))&(user_taggedmovies_timestamps['movieID'].isin(frequentItem))].sort_values(by=['timestamp'])
userFlow=list(freqData.userID)
trueResultFlow=list(freqData.movieID)
len(userFlow)

In [ ]:
import jieba  
import jieba.posseg as pseg  
import os  
import sys 
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

corpus=list(tags.value)
vectorizer=CountVectorizer()
transformer=TfidfTransformer()  
tfidf=transformer.fit_transform(vectorizer.fit_transform(corpus)) 
word=vectorizer.get_feature_names()
weight=tfidf.toarray()
print( 'Start Kmeans:')

clf = KMeans(n_clusters=4)   
s = clf.fit(weight)
label = []     
i = 1
while i <= len(clf.labels_):
    #print( clf.labels_[i-1])
    label.append(clf.labels_[i-1])
    i = i + 1


pca = PCA(n_components=10)
newData = pca.fit_transform(weight)  

In [ ]:
np.save(r'newData.npy',newData)
newData=np.load(r'newData.npy')

In [ ]:
newDataDict={}
for i in range(len(newData)):
    newDataDict[tags.id[i]]=newData[i]

frequentItemDict={}
for i in frequentItem:
    temp=np.zeros(10)
    for j in list(user_taggedmovies_timestamps[user_taggedmovies_timestamps.movieID==i].tagID):
        if j<len(newData):
            temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0.:
        print(i)
        frequentItemDict[i]=temp
    else:
        frequentItemDict[i]=temp/np.linalg.norm(temp, ord=2)

In [ ]:
clf1 = KMeans(n_clusters=40)
s1 = clf1.fit(list(frequentItemDict.values()))

frequentItemCluster={}
for i in range(len(frequentItem)):
    frequentItemCluster[frequentItem[i]]=clf1.labels_[i]
frequentItemVec={}
for i in range(40):
    frequentItemVec[i]=np.zeros(10)
for i in frequentItem:
    frequentItemVec[frequentItemCluster[i]]+=frequentItemDict[i]
for i in range(40):
    frequentItemVec[i]=frequentItemVec[i]/np.linalg.norm(frequentItemVec[i], ord=2)

stage1Data=user_taggedmovies_timestamps
#stage2Data=user_taggedmovies_timestamps[50000:100000]
#stage3Data=user_taggedmovies_timestamps[100000:150000]
#stage4Data=user_taggedmovies_timestamps[150000:]
frequentUserDict1={}
#frequentUserDict2={}
#frequentUserDict3={}
#frequentUserDict4={}
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage1Data[stage1Data.userID==i].tagID):
        if j<len(newData):
            temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict1[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict1[i]=temp/np.linalg.norm(temp, ord=2)
        
"""
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage2Data[stage2Data.userID==i].id):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict2[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict2[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage3Data[stage3Data.userID==i].id):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict3[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict3[i]=temp/np.linalg.norm(temp, ord=2)
for i in frequentUser:
    temp=np.zeros(10)
    for j in list(stage4Data[stage4Data.userID==i].id):
        temp+=newDataDict[j]
    if np.linalg.norm(temp, ord=2)==0:
        frequentUserDict4[i]=np.ones(10)/np.linalg.norm(np.ones(10), ord=2)
    else:
        frequentUserDict4[i]=temp/np.linalg.norm(temp, ord=2)

freqUserFeatureDict={}
for i in frequentUser:
    freqUserFeatureDict[i]=[]
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict1[i])
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict2[i])
    for j in range(len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict3[i])
    for j in range(len(freqData)-3*len(freqData)//4):
        freqUserFeatureDict[i].append(frequentUserDict4[i])
"""


In [ ]:
np.save(r'frequentUserDict300_del.npy',frequentUserDict1)
np.save(r'frequentItemVec.npy',frequentItemVec)

In [ ]:
indexflow=[]
for i in range(len(user_taggedmovies_timestamps)):
    if i%500==0:
        print(i)
    if user_taggedmovies_timestamps.iloc[i].movieID in list(frequentItemDict.keys()):
        indexflow.append(i)
movieflow=user_taggedmovies_timestamps.iloc[indexflow].movieID.values
userflow=user_taggedmovies_timestamps.iloc[indexflow].userID.values
movieflow=[frequentItemCluster[i] for i in movieflow]
np.save(r'userflow.npy',userflow)
np.save(r'movieflow.npy',movieflow)
np.save(r'frequentItemVec.npy',np.array(list(frequentItemVec.values())))

In [ ]:
np.save(r'userflow.npy',userflow)
np.save(r'movieflow.npy',movieflow)
np.save(r'frequentItemVec.npy',np.array(list(frequentItemVec.values())))

In [ ]:
userflowdict={}
userflowlist=list(set(userflow))
for i in range(len(set(userflow))):
    userflowdict[userflowlist[i]]=i
userclick=[[] for i in range(len(set(userflow)))]
for i in range(len(userflow)):
    userclick[userflowdict[userflow[i]]].append(movieflow[i])
userflowRenew=[]
for i in userflow:
    userflowRenew.append(userflowdict[i])

In [ ]:
np.save(r'userclick.npy',userclick)
np.save(r'userflow.npy',userflowRenew)